In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn import metrics

df = pd.read_csv('../data/archive', header=0, delimiter=',')
df['category_id'] = df['Category'].factorize()[0]
df = df.drop_duplicates(subset='website_url').reset_index(drop=True)
df = df.dropna()



,Unnamed: 0,website_url,cleaned_website_text,Category,category_id
0,0,https://www.booking.com/index.html?aid=1743217,official site good hotel accommodation big sav...,Travel,0
1,1,https://travelsites.com/expedia/,expedia hotel book sites like use vacation wor...,Travel,0
2,2,https://travelsites.com/tripadvisor/,tripadvisor hotel book sites like previously d...,Travel,0
3,3,https://www.momondo.in/?ispredir=true,cheap flights search compare flights momondo f...,Travel,0
4,4,https://www.ebookers.com/?AFFCID=EBOOKERS-UK.n...,bot create free account create free account si...,Travel,0


In [22]:

df.isnull().sum()

Unnamed: 0              0
website_url             0
cleaned_website_text    0
Category                0
category_id             0
dtype: int64

In [23]:
#tfidf= TfidfVectorizer(sublinear_tf=True, min_df=6, ngram_range=(1, 2), stop_words='english')
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=6, ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.cleaned_website_text).toarray()
labels = df.category_id

X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               df.index, test_size=0.3, 
                                                               random_state=1)

model = LinearSVC()
model.fit(X_train, y_train)

calibrated_svc = CalibratedClassifierCV(base_estimator=model, cv="prefit")
calibrated_svc.fit(X_train,y_train)
predicted = calibrated_svc.predict(X_test)
print("Test Accuracy LinearSVC: %.3f" % metrics.accuracy_score(y_test, predicted))

Test Accuracy LinearSVC: 0.939
